In [6]:
import os 

import numpy as np
import pandas as pd

from pathlib import Path
from glob import glob
from tqdm import tqdm

Loading data from train.jsonl file into chunks of parquet files

In [7]:
%%time
#data_path = Path('/kaggle/input/otto-recommender-system/')
chunksize = 100_000
save = True

type_map = {
    "clicks": 1,
    "carts": 3,
    "orders": 5
}

chunks = pd.read_json('data/train.jsonl', lines=True, chunksize=chunksize)
#os.mkdir('data/train_parquet')

for e, chunk in enumerate(tqdm(chunks, total=129)):
    event_dict = {
        'session': [],
        'aid': [],
        'type': [],
    }
    
    for session, events in zip(chunk['session'].tolist(), chunk['events'].tolist()):
        for event in events:
            event_dict['session'].append(session)
            event_dict['aid'].append(event['aid'])
            #event_dict['ts'].append(event['ts'])
            #event_dict['type'].append(event['type'])
            event_dict['type'].append(type_map[event['type']])
    
    # save DataFrame
    start = str(e*chunksize).zfill(9)
    end = str(e*chunksize+chunksize).zfill(9)
    event_df = pd.DataFrame(event_dict)
    if save == True:
        event_df.to_parquet(f"data/train_parquet/{start}_{end}.parquet")

100%|█████████████████████████████████████████| 129/129 [11:25<00:00,  5.32s/it]

CPU times: user 11min, sys: 17.3 s, total: 11min 18s
Wall time: 11min 25s


Loading data from test.jsonl into parquets

In [11]:
%%time
#data_path = Path('/kaggle/input/otto-recommender-system/')
chunksize = 100_000
save = True

type_map = {
    "clicks": 1,
    "carts": 3,
    "orders": 5
}

chunks = pd.read_json('data/test.jsonl', lines=True, chunksize=chunksize)
#os.mkdir('data/train_parquet')
x=129
for e, chunk in enumerate(tqdm(chunks)):
    event_dict = {
        'session': [],
        'aid': [],
        'type': [],
    }
    
    for session, events in zip(chunk['session'].tolist(), chunk['events'].tolist()):
        for event in events:
            event_dict['session'].append(session)
            event_dict['aid'].append(event['aid'])
            #event_dict['ts'].append(event['ts'])
            #event_dict['type'].append(event['type'])
            event_dict['type'].append(type_map[event['type']])
    
    # save DataFrame
    start = str(x*chunksize).zfill(9)
    end = str(x*chunksize+chunksize).zfill(9)
    x=x+1
    event_df = pd.DataFrame(event_dict)
    if save == True:
        event_df.to_parquet(f"data/train_parquet/{start}_{end}.parquet")

17it [00:32,  1.94s/it]

CPU times: user 32.5 s, sys: 519 ms, total: 33 s
Wall time: 32.9 s


Loading and displaying a single parquet file

In [4]:
%%time
event_df = pd.read_parquet('data/train_parquet/000000000_000100000.parquet')
event_df.head()

CPU times: user 128 ms, sys: 117 ms, total: 245 ms
Wall time: 131 ms


,session,aid,type
0,0,1517085,1
1,0,1563459,1
2,0,1309446,1
3,0,16246,1
4,0,1781822,1


In [6]:
event_df.dtypes

session    int64
aid        int64
type       int64
dtype: object